In [7]:
import shutil
import os
import xml_to_dict as xtd
import random


# Process CVAT labelled data for YOLOv8 training

The provided data by ASPL was labelled through rotated bounding box. However, rotated/oriented bounding box remains to be a frontier of research and there is not a lot of format supporting it.

Thus, this is an attempt to convert from LabelMe format to YOLO OBB/Segmentation format.


In [ ]:
def process_cvat():
    search_dir = os.path.abspath('../../datasets/aspl_images_post_labelled\images')

    obj_idx_dict = {
            "Main Pad": 0,
            "LED Pads": 1,
            "Main Chip": 2,
            "Left LED Chip": 3,
            "Right LED Chip": 4,
            "Main PCB": 5
        }

    parser = xtd.XMLtoDict()

    for root, _, files in os.walk(search_dir):

        for file in files:

            if file.endswith(".xml"):

                file_path = os.path.join(root, file)
                file_name = file.split(".")[0]

                with open(file_path, 'r') as f:
                    content = f.read()
                
                xml_dict = parser.parse(content)['annotation']

                img_x = int(xml_dict['imagesize']['ncols'])
                img_y = int(xml_dict['imagesize']['nrows'])

                with open(os.path.join(root, file_name + '.txt'), 'w') as f:

                    for obj in xml_dict['object']:
                        obj_name = obj['name']
                        verts = []

                        for pt in obj['polygon']['pt']:
                            verts.append(f"{float(pt['x']) / img_x:.6f}")
                            verts.append(f"{float(pt['y']) / img_y:.6f}")
                    
                        f.write(f'{str(obj_idx_dict[obj_name])} {" ".join(verts)}\n')
    
process_cvat()


In [8]:
def split_cvat():
    search_dir = os.path.abspath('../../datasets/aspl_images_post_labelled/images/Train')
    val_dir = os.path.abspath('../../datasets/aspl_images_post_labelled/images/Val')
    
    if not os.path.exists(val_dir):
        os.makedirs(val_dir)

    for file in os.listdir(search_dir):
        if file.endswith(".txt"):

            file_name = file.split(".")[0]

            if random.random() < 0.2:
                shutil.move(os.path.join(search_dir, file), val_dir)
                shutil.move(os.path.join(search_dir, file_name + '.jpg'), val_dir)

split_cvat()


# Process Rendered data for YOLOv8 training

Rendered data is already compatible with YOLO OBB/Segmentation format.

Just get a list of all files, and copy both the label and the image into train/test folders.


In [ ]:
def process_rendered():
    img_dir = os.path.abspath('../../datasets/rendered')
    lbl_dir = os.path.abspath('../../datasets/rendered/labels')

    val_dir = os.path.abspath('../../datasets/rendered/val')
    train_dir = os.path.abspath('../../datasets/rendered/train')

    if not os.path.exists(val_dir):
        os.makedirs(val_dir)

    if not os.path.exists(train_dir):
        os.makedirs(train_dir)

    file_list = os.listdir(img_dir)
    n_files = len(file_list)

    for i, item in enumerate(file_list):
        
        if item.endswith('.png'):
            item_name = item.split('.')[0]
            img_path = os.path.join(img_dir, item)
            lbl_path = os.path.join(lbl_dir, item_name + '.txt')

            if random.random() < 0.2:
                shutil.move(img_path, os.path.join(val_dir, item))
                shutil.move(lbl_path, os.path.join(val_dir, item_name + '.txt'))
            else:
                shutil.move(img_path, os.path.join(train_dir, item))
                shutil.move(lbl_path, os.path.join(train_dir, item_name + '.txt'))

        if (i - 1) % 100 == 0 or i == n_files - 1:
            print(f"{i+1} / {n_files} files processed")
    
process_rendered()
